<a href="https://colab.research.google.com/github/Shorya777/Neural_Machine_Translation_from_scratch/blob/master/research/trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d parvmodi/english-to-hindi-machine-translation-dataset

Dataset URL: https://www.kaggle.com/datasets/parvmodi/english-to-hindi-machine-translation-dataset
License(s): CC0-1.0
 99% 1.03G/1.04G [00:20<00:00, 88.2MB/s]
100% 1.04G/1.04G [00:20<00:00, 54.3MB/s]


In [2]:
!unzip english-to-hindi-machine-translation-dataset.zip

Archive:  english-to-hindi-machine-translation-dataset.zip
  inflating: train.en                
  inflating: train.hi                


In [3]:
filepath_en = 'train.en'
filepath_hi = 'train.hi'

with open(filepath_en, encoding='utf-8') as file:
    lines_en = file.readlines()
print(len(lines_en))
print(lines_en[:5])

with open(filepath_hi, encoding='utf-8') as file:
    lines_hi = file.readlines()

print(len(lines_hi))
print(lines_hi[:5])

10125706
["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles\n", 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.\n', 'The value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.\n', 'Mithali To Anchor Indian Team Against Australia in ODIs\n', 'After the assent of the Honble President on 8thSeptember, 2016, the 101thConstitutional Amendment Act, 2016 came into existence\n']
10125706
['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।\n', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है\n', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प

In [4]:
TOTAL_SENTENCES = 200000
english_sentences = lines_en[:TOTAL_SENTENCES]
hindi_sentences = lines_hi[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

print(english_sentences[:5])
print("================================")
print(hindi_sentences[:5])

["however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles", 'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.', 'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.', 'mithali to anchor indian team against australia in odis', 'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence']
['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले मे

In [5]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 258.0
97th percentile length English: 267.0


In [6]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\' ', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

hindi_vocabulary = [START_TOKEN, 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ',
    'क', 'ख', 'ग', 'घ', 'ङ','ड़', 'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
    'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो', 'ौ',
    'ं', 'ः', 'ँ', '्', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '|',
    ':', '<', '=', '>', '?', '@',
    '[', '\' ', ']', '^', '_', '`',
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','\u200d','।', 'ृ',
    PADDING_TOKEN, END_TOKEN
]

print(len(english_vocabulary))

71


In [7]:
max_sequence_length = 250

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")


Number of sentences: 200000
Number of valid sentences: 151260


In [8]:
# for i in 'शख्स':
  # print(i)


# hindi_sentences = [
#     'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
#     'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
#     'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।',
#     'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
#     '8 सितम्‍बर, 2016 को माननीय राष्ट्रपति की स्‍वीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्व में आया'
# ]

# Define the provided vocabulary set
hindi_vocabulary_set = set(hindi_vocabulary)

# Combine all sentences into one string
combined_sentences = ''.join(hindi_sentences)

# Find all unique characters in the combined sentences
unique_chars_in_sentences = set(combined_sentences)

# Find characters in the sentences that are not in the vocabulary
missing_chars = unique_chars_in_sentences - hindi_vocabulary_set

print("Characters in the sentences not in the vocabulary:", missing_chars)


Characters in the sentences not in the vocabulary: {'H', 'ְ', '™', '♦', 'ˌ', 'ಳ', 'ා', 'x', 'ö', 'ɵ', 'ต', 'ς', '․', 'ع', 'λ', 'ऌ', 'ॅ', 'ু', 'ច', '॥', 'è', '್', '□', '′', '章', 'M', 'و', 'コ', 'X', 'à', '≈', '\uf0d8', '˜', 'ط', '西', 'ă', '\ue208', 'ড', 'з', 'ف', 'ī', 'ల', 'М', '班', 'ч', '~', 'ு', 'ක', ';', 'w', 'ො', '੍', '“', 'ί', 'ದ', 's', 'জ', '\ue5a0', 'ನ', 'б', 'ម', '£', 'র', 'j', 'ে', 'ы', '공', 'ॄ', 'ᒡ', 'ය', '්', '−', 't', 'ー', '記', '經', 'ห', 'ុ', 'q', '§', '☞', 'ว', 'כ', '한', '♪', 'ි', 'ॠ', 'Ň', 'ʔ', 'ৱ', 'パ', '女', 'ோ', 'ē', 'þ', 'ਦ', 'ุ', '°', 'ּ', 'ǔ', 'ॉ', 'Я', 'r', 'ย', 'ন', '\uf146', 'Э', 'ำ', 'ை', 'ó', 'ਤ', 'ඩ', 'Ż', '日', 'ę', 'ු', 'ʁ', 'â', 'ស', 'ර', 'க', 'ऱ', '½', 'c', '☺', 'េ', 'Z', 'ল', 'ס', 'క', 'த', 'י', 'ಿ', '›', 'é', 'ត', 'R', '◆', '਼', '্', '의', 'ص', '்', 'υ', 'ғ', 'ऽ', 'ä', '❑', 'S', 'ල', 'ָ', '\ue205', '郑', 'ன', 'Y', 'พ', 'æ', '–', '¶', 'শ', '夕', '́', 'ও', 'J', 'ಕ', 'o', 'ت', 'у', 'ש', 'ش', 'ি', 'F', 'ו', 'ᡠ', 'Ê', 'Ю', '្', 'ං', '©', '¡', '代', 'แ', 'م', 'р', 'ਆ'

In [9]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}
print(english_to_index)

{'<START>': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, '<': 28, '=': 29, '>': 30, '?': 31, '@': 32, '[': 33, "' ": 34, ']': 35, '^': 36, '_': 37, '`': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64, '{': 65, '|': 66, '}': 67, '~': 68, '<PADDING>': 69, '<END>': 70}


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras import layers
import numpy as np

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, max_seq_len):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

    def call(self, inputs):
        pos = np.arange(self.max_seq_len)[:, np.newaxis]
        i = np.arange(self.d_model)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.d_model))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.constant(pos_encoding, dtype=tf.float32)

class SentenceEmbedding(tf.keras.Model):
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super(SentenceEmbedding, self).__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = Dropout(0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN

    def batch_tokenize(self, batch, start_token, end_token):
        def tokenize(sentence, start_token, end_token):
            sentence_word_indices = [self.language_to_index[token] for token in list(sentence.numpy().decode('utf-8'))]
            if start_token:
                sentence_word_indices.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indices.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indices), self.max_sequence_length):
                sentence_word_indices.append(self.language_to_index[self.PADDING_TOKEN])
            return tf.convert_to_tensor(sentence_word_indices, dtype=tf.int32)

        # if not isinstance(batch, (list, tuple)):
        #     batch = [batch]  # Convert to a list if it's a single tensor
        tokenized = [tokenize(sentence, start_token, end_token) for sentence in batch]
        return tf.stack(tokenized)

    def call(self, x, start_token, end_token):
        x = self.batch_tokenize(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder(x)
        x = self.dropout(x + pos)
        return x

In [11]:
def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_english_sentences = []
valid_hindi_sentences = []
for index in valid_sentence_indicies:
    valid_english_sentences.append(english_sentences[index])
    valid_hindi_sentences.append(hindi_sentences[index])

def batch_tokenize(language_to_index, batch, start_token, end_token, START_TOKEN, END_TOKEN, PADDING_TOKEN, max_sequence_length):
    def tokenize(sentence, start_token, end_token):
        sentence_word_indices = [language_to_index[token] for token in list(sentence)]
        if start_token:
            sentence_word_indices.insert(0, language_to_index[START_TOKEN])
        if end_token:
            sentence_word_indices.append(language_to_index[END_TOKEN])
        if max_sequence_length > len(sentence_word_indices):
            sentence_word_indices.extend([language_to_index[PADDING_TOKEN]] * (max_sequence_length - len(sentence_word_indices)))
        return tf.convert_to_tensor(sentence_word_indices, dtype=tf.int32)

    tokenized = [tokenize(sentence, start_token, end_token) for sentence in batch]
    return tf.stack(tokenized)

x = batch_tokenize(language_to_index = english_to_index, batch = valid_english_sentences[:10], start_token = True, end_token = True,
                     START_TOKEN= START_TOKEN, END_TOKEN= END_TOKEN, PADDING_TOKEN=PADDING_TOKEN, max_sequence_length= max_sequence_length)

print(x)

tf.Tensor(
[[ 0 46 53 ... 69 69 69]
 [ 0 61 46 ... 69 69 69]
 [ 0 58 46 ... 69 69 69]
 ...
 [ 0 48 46 ... 69 69 69]
 [ 0 39 56 ... 69 69 69]
 [ 0 40 56 ... 69 69 69]], shape=(10, 250), dtype=int32)


In [12]:
import tensorflow as tf

d_model = 512

sentence_embedding = SentenceEmbedding(
    max_sequence_length=max_sequence_length,
    d_model=d_model,
    language_to_index=english_to_index,
    START_TOKEN=START_TOKEN,
    END_TOKEN=END_TOKEN,
    PADDING_TOKEN=PADDING_TOKEN
)

# output = sentence_embedding(valid_english_sentences[:10], start_token=True, end_token=True)

# print(output)

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Layer
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = tf.shape(q)[-1]
    scaled = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(d_k, tf.float32))
    if mask is not None:
        scaled = tf.transpose(scaled, perm=[1, 0, 2, 3])

        # Add the mask (broadcasting it over the head and batch dimensions)
        mask = tf.expand_dims(mask, axis=0)  # Expand dims to match head dimension
        scaled += mask

        # Permute back to the original shape
        scaled = tf.transpose(scaled, perm=[1, 0, 2, 3])

    attention = tf.nn.softmax(scaled, axis=-1)
    values = tf.matmul(attention, v)
    return values, attention

class MultiHeadAttention(Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = Dense(3 * d_model)
        self.linear_layer = Dense(d_model)

    def call(self, x, mask=None):
        batch_size = tf.shape(x)[0]
        sequence_length = tf.shape(x)[1]
        qkv = self.qkv_layer(x)
        qkv = tf.reshape(qkv, (batch_size, sequence_length, self.num_heads, 3 * self.head_dim))
        qkv = tf.transpose(qkv, perm=[0, 2, 1, 3])
        q, k, v = tf.split(qkv, num_or_size_splits=3, axis=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = tf.transpose(values, perm=[0, 2, 1, 3])
        values = tf.reshape(values, (batch_size, sequence_length, self.num_heads * self.head_dim))
        out = self.linear_layer(values)
        return out

num_heads= 8
multihead_attention = MultiHeadAttention(d_model= d_model, num_heads= num_heads)

# x = multihead_attention(x= output)
# x

In [16]:
class LayerNormalization(tf.keras.layers.Layer):
    def __init__(self, parameters_shape, eps=1e-5):
        super(LayerNormalization, self).__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = self.add_weight(
            shape=parameters_shape,
            initializer="ones",
            trainable=True,
            name="gamma"
        )
        self.beta = self.add_weight(
            shape=parameters_shape,
            initializer="zeros",
            trainable=True,
            name="beta"
        )

    def call(self, inputs):
        dims = [-i for i in range(1, len(self.parameters_shape) + 1)]
        mean = tf.reduce_mean(inputs, axis=dims, keepdims=True)
        var = tf.reduce_mean(tf.square(inputs - mean), axis=dims, keepdims=True)
        std = tf.sqrt(var + self.eps)
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out


class PositionwiseFeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = tf.keras.layers.Dense(hidden)
        self.linear2 = tf.keras.layers.Dense(d_model)
        self.relu = tf.keras.layers.ReLU()
        self.dropout = tf.keras.layers.Dropout(rate=drop_prob)

    def call(self, x, training=False):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x, training=training)
        x = self.linear2(x)
        return x


class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = tf.keras.layers.Dropout(rate=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = tf.keras.layers.Dropout(rate=drop_prob)

    def call(self, x, self_attention_mask, training=False):
        residual_x = x
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x, training=training)
        x = self.norm1(x + residual_x)
        residual_x = x
        x = self.ffn(x, training=training)
        x = self.dropout2(x, training=training)
        x = self.norm2(x + residual_x)
        return x

num_heads =8
ffn_hidden = 2048
encoder_layer = EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob =0.1)

# x = encoder_layer(x,self_attention_mask= None)
# x

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices(valid_english_sentences)
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
iterator = iter(train_dataset)
first_batch = next(iterator)
# first_batch = tf.reshape(first_batch, (1,batch_size))
first_batch = [sentence.numpy().decode('utf-8') for sentence in first_batch]
print(first_batch)

["however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles", 'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.', 'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.', 'mithali to anchor indian team against australia in odis', 'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence', 'the court has fixed a hearing for february 12', 'please select the position where the track should be split.', 'jharkhand chief minister hemant soren', 'arvind kumar, sho of the sector 55/56 police station, said a case has been registered under section 376-d (gang rape) of the indian penal code.', "briefing media in new delhi today, party's state-in-charge, anil jain said, after the me

In [18]:
class SequentialEncoder(tf.keras.Sequential):
    def call(self, inputs, self_attention_mask, training=False):
        x = inputs
        for module in self.layers:
            x = module(x, self_attention_mask, training=training)
        return x


class Encoder(tf.keras.layers.Layer):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super(Encoder, self).__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder([EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def call(self, x, self_attention_mask, start_token, end_token, training=False):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask, training=training)
        return x

    def get_layers(self):
        return self.layers

drop_prob =0.1
encoder = Encoder(d_model =d_model,ffn_hidden=  ffn_hidden,num_heads= num_heads, drop_prob= drop_prob, num_layers= 1, max_sequence_length= max_sequence_length,
                  language_to_index= english_to_index, START_TOKEN= START_TOKEN, END_TOKEN= END_TOKEN, PADDING_TOKEN= PADDING_TOKEN)
# x = encoder(x= first_batch, self_attention_mask= None, start_token= True, end_token= True)
# x

In [19]:
class MultiHeadCrossAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadCrossAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = Dense(2 * d_model)
        self.q_layer = Dense(d_model)
        self.linear_layer = Dense(d_model)

    def call(self, x, y, mask):
        batch_size, sequence_length, d_model = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = tf.reshape(kv, (batch_size, sequence_length, self.num_heads, 2 * self.head_dim))
        q = tf.reshape(q, (batch_size, sequence_length, self.num_heads, self.head_dim))
        kv = tf.transpose(kv, perm=[0, 2, 1, 3])
        q = tf.transpose(q, perm=[0, 2, 1, 3])
        k, v = tf.split(kv, 2, axis=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = tf.transpose(values, perm=[0, 2, 1, 3])
        values = tf.reshape(values, (batch_size, sequence_length, d_model))
        out = self.linear_layer(values)
        return out

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(num_heads=num_heads, d_model=d_model)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = Dropout(drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = Dropout(drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = Dropout(drop_prob)

    def call(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y

    # def get_sub_layers(self):
        # return [self.self_attention, self.encoder_decoder_attention, self.ffn, self.layer_norm1, self.layer_norm2, self.layer_norm3]



class SequentialDecoder(tf.keras.Sequential):
    def call(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for layer in self.layers:
            y = layer(x, y, self_attention_mask, cross_attention_mask)
        return y


class Decoder(tf.keras.layers.Layer):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super(Decoder, self).__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder([DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])
        # self.layers = tf.keras.Sequential([DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])


    def call(self, x, y, self_attention_mask=None, cross_attention_mask=None, start_token=True, end_token=True):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

    def get_layers(self):
        return self.layers


class Transformer(tf.keras.layers.Layer):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 hi_vocab_size,
                 english_to_index,
                 hindi_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super(Transformer, self).__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, hindi_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = Dense(hi_vocab_size)
        self.device = tf.config.experimental.list_physical_devices('GPU')

    def call(self,
             x,
             y,
             encoder_self_attention_mask=None,
             decoder_self_attention_mask=None,
             decoder_cross_attention_mask=None,
             enc_start_token=False,
             enc_end_token=False,
             dec_start_token=False, # We should make this true
             dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        print("encoder done")
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        print("decoder done")
        out = self.linear(out)
        return out

In [20]:
print(valid_hindi_sentences[:10])
print(valid_english_sentences[:10])

['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।', 'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को', '8 सितम्\u200dबर, 2016 को माननीय राष्\u200dट्रपति की स्\u200dवीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्\u200dव में आया', 'अदालत ने इस मामले में आगे की सुनवाई के लिए एक फरवरी की तारीख़ तय की', 'जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया वह स्थान चुनें.', 'झारखंड के मुख्यमंत्री हेमंत सोरेन (फोटोः पीटीआई)', 'सेक्टर 55/56 के एसएचओ अरविंद कुमार ने बताया कि इस मामले में आईपीसी की धारा 376-डी (गैंगरेप) के तहत मामला दर्ज कर लिया गया है।', 'आज नई दिल्ली में मीडिया से बातचीत में पार्टी के राज्य 

In [21]:
import tensorflow as tf

class TextDataset:
    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]



dataset = TextDataset(valid_english_sentences, valid_hindi_sentences)

def generator():
    for i in range(len(dataset)):
        eng_sentence, hi_sentence = dataset[i]
        yield eng_sentence, hi_sentence

output_signature = (
    tf.TensorSpec(shape=(), dtype=tf.string),
    tf.TensorSpec(shape=(), dtype=tf.string)
)

tf_dataset = tf.data.Dataset.from_generator(generator, output_signature=output_signature)

# Define batch size
batch_size = 32

# Create a batched dataset
tf_dataset = tf_dataset.batch(batch_size)

# Create an iterator
iterator = iter(tf_dataset)

# Iterate through the dataset

for batch_num, batch in enumerate(iterator):
    print(batch[0])
    if batch_num > -1:
        break


tf.Tensor(
[b"however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles"
 b'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.'
 b'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.'
 b'mithali to anchor indian team against australia in odis'
 b'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence'
 b'the court has fixed a hearing for february 12'
 b'please select the position where the track should be split.'
 b'jharkhand chief minister hemant soren'
 b'arvind kumar, sho of the sector 55/56 police station, said a case has been registered under section 376-d (gang rape) of the indian penal code.'
 b"briefing media in new delhi today, party's state-in-charge, anil ja

In [22]:
sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
sentence_embedding.batch_tokenize(next(iterator)[0], start_token= True, end_token= True)
# def batch_tokenize(batch, start_token, end_token):
#     def tokenize(sentence, start_token, end_token):
#         sentence_word_indices = [english_to_index[token] for token in list(sentence.numpy().decode('utf-8'))]
#         if start_token:
#             sentence_word_indices.insert(0, english_to_index[start_token])
#         if end_token:
#             sentence_word_indices.append(english_to_index[end_token])
#         for _ in range(len(sentence_word_indices), max_sequence_length):
#             sentence_word_indices.append(english_to_index[PADDING_TOKEN])
#         return tf.convert_to_tensor(sentence_word_indices, dtype=tf.int32)

#     # if not isinstance(batch, (list, tuple)):
#         # batch = [batch]  # Convert to a list if it's a single tensor
#     tokenized = [tokenize(sentence, start_token, end_token) for sentence in batch]
#     return tf.stack(tokenized)

# for sentence in next(iterator)[0]:
    # sentence_word_indices = [english_to_index[token] for token in list(sentence.numpy().decode('utf-8'))]
    # print(len(sentence.numpy().decode('utf-8')))
    # print(len(sentence_word_indices))
    # print(sentence)

# batch_tokenize(next(iterator)[0], START_TOKEN, END_TOKEN)

<tf.Tensor: shape=(32, 250), dtype=int32, numpy=
array([[ 0, 46, 47, ..., 69, 69, 69],
       [ 0, 58, 46, ..., 69, 69, 69],
       [ 0, 61, 47, ..., 69, 69, 69],
       ...,
       [ 0, 40, 59, ..., 69, 69, 69],
       [ 0, 25, 25, ..., 69, 69, 69],
       [ 0, 43, 60, ..., 69, 69, 69]], dtype=int32)>

In [ ]:
# import tensorflow as tf

# class TextDataset(tf.data.Dataset):
#     def _generator(english_sentences, hindi_sentences):
#         for eng, hi in zip(english_sentences, hindi_sentences):
#             yield eng, hi

#     def __new__(cls, english_sentences, hindi_sentences):
#         return tf.data.Dataset.from_generator(
#             cls._generator,
#             output_signature=(
#                 tf.TensorSpec(shape=(), dtype=tf.string),
#                 tf.TensorSpec(shape=(), dtype=tf.string)
#             ),
#             args=(english_sentences, hindi_sentences)
#         )


# dataset = TextDataset(valid_english_sentences, valid_hindi_sentences)
# dataset = dataset.batch(batch_size)


# batches_list = []
# for batch in dataset:
#     batch_list = []
#     batch_list_eng = []
#     batch_list_kan = []
#     for eng, kan in zip(batch[0], batch[1]):
#         batch_list_eng.append(eng.numpy().decode("utf-8"))
#         batch_list_kan.append(kan.numpy().decode("utf-8"))
#     batch_list.append(tuple(batch_list_eng))
#     batch_list.append(tuple(batch_list_kan))
#     batches_list.append(batch_list)

# batches_list[0]

In [ ]:
encoder = Encoder(d_model =d_model,ffn_hidden=  ffn_hidden,num_heads= num_heads, drop_prob= drop_prob, num_layers= 1, max_sequence_length= max_sequence_length,
                  language_to_index= english_to_index, START_TOKEN= START_TOKEN, END_TOKEN= END_TOKEN, PADDING_TOKEN= PADDING_TOKEN)
for batch_num, batch in enumerate(iterator):
    # x = encoder(x= batch[0], self_attention_mask= None, start_token= True, end_token= True)
    for sentence in batch[0]:
      print(sentence.numpy().decode('utf-8'))
    if batch_num > -1:
        break

In [23]:
d_model = 512
batch_size = 32
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
hi_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          hi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [24]:
initializer = tf.keras.initializers.GlorotUniform()

# Iterate over the model's layers and initialize weights with Xavier uniform initializer
for layer in transformer.encoder.get_layers().layers:
    if isinstance(layer, tf.keras.layers.Dense):
        layer.kernel_initializer = initializer
        # To reinitialize the kernel with the new initializer
        layer.build(layer.input_shape)
        print(layer)

for layer in transformer.decoder.get_layers().layers:
    if isinstance(layer, tf.keras.layers.Dense):
        layer.kernel_initializer = initializer
        # To reinitialize the kernel with the new initializer
        layer.build(layer.input_shape)

In [25]:
import tensorflow as tf
import numpy as np

NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch, max_sequence_length):
    # eng_batch = eng_batch.numpy()
    # kn_batch = kn_batch.numpy()
    num_sentences = tf.shape(eng_batch)[0]

    # Look-ahead mask for decoder
    look_ahead_mask = tf.linalg.band_part(tf.ones((max_sequence_length, max_sequence_length)), -1, 0)
    look_ahead_mask = 1 - look_ahead_mask

    encoder_padding_mask = tf.zeros((num_sentences, max_sequence_length, max_sequence_length), dtype=tf.float32)
    decoder_padding_mask_self_attention = tf.zeros((num_sentences, max_sequence_length, max_sequence_length), dtype=tf.float32)
    decoder_padding_mask_cross_attention = tf.zeros((num_sentences, max_sequence_length, max_sequence_length), dtype=tf.float32)

    for idx in range(num_sentences):
        eng_sentence_length = len(eng_batch[idx].numpy().decode('utf-8'))
        kn_sentence_length = len(kn_batch[idx].numpy().decode('utf-8'))

        eng_chars_to_padding_mask = tf.range(eng_sentence_length + 1, max_sequence_length)
        # print(kn_sentence_length)
        # print(kn_batch[idx].decode('utf-8'))
        kn_chars_to_padding_mask = tf.range(kn_sentence_length + 1, max_sequence_length)

        # Update encoder padding mask
        for pos in eng_chars_to_padding_mask:
            encoder_padding_mask = tf.tensor_scatter_nd_update(
                encoder_padding_mask,
                [[idx, pos, i] for i in range(max_sequence_length)],
                tf.ones((max_sequence_length,), dtype=tf.float32)
            )
            encoder_padding_mask = tf.tensor_scatter_nd_update(
                encoder_padding_mask,
                [[idx, i, pos] for i in range(max_sequence_length)],
                tf.ones((max_sequence_length,), dtype=tf.float32)
            )

        # Update decoder padding mask for self-attention
        for pos in kn_chars_to_padding_mask:
            decoder_padding_mask_self_attention = tf.tensor_scatter_nd_update(
                decoder_padding_mask_self_attention,
                [[idx, pos, i] for i in range(max_sequence_length)],
                tf.ones((max_sequence_length,), dtype=tf.float32)
            )
            decoder_padding_mask_self_attention = tf.tensor_scatter_nd_update(
                decoder_padding_mask_self_attention,
                [[idx, i, pos] for i in range(max_sequence_length)],
                tf.ones((max_sequence_length,), dtype=tf.float32)
            )

        # Update decoder padding mask for cross-attention
        for pos in eng_chars_to_padding_mask:
            decoder_padding_mask_cross_attention = tf.tensor_scatter_nd_update(
                decoder_padding_mask_cross_attention,
                [[idx, pos, i] for i in range(max_sequence_length)],
                tf.ones((max_sequence_length,), dtype=tf.float32)
            )
        for pos in kn_chars_to_padding_mask:
            decoder_padding_mask_cross_attention = tf.tensor_scatter_nd_update(
                decoder_padding_mask_cross_attention,
                [[idx, i, pos] for i in range(max_sequence_length)],
                tf.ones((max_sequence_length,), dtype=tf.float32)
            )

    encoder_self_attention_mask = tf.where(encoder_padding_mask>0, NEG_INFTY, 0.0)
    decoder_self_attention_mask = tf.where(tf.cast(look_ahead_mask, tf.float32) + decoder_padding_mask_self_attention > 0, NEG_INFTY, 0.0)
    decoder_cross_attention_mask = tf.where(decoder_padding_mask_cross_attention > 0, NEG_INFTY, 0.0)

    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask


# look_ahead_mask = tf.linalg.band_part(tf.ones((max_sequence_length, max_sequence_length)), -1, 0)
# look_ahead_mask = 1 - look_ahead_mask #normal mask taught

# eng_chars_to_padding_mask = np.arange( 50+ 1, max_sequence_length)
# encoder_padding_mask = tf.tensor_scatter_nd_update(encoder_padding_mask,
#                                                     indices=[[idx, :, eng_chars_to_padding_mask]],
#                                                     updates=tf.ones_like(eng_chars_to_padding_mask, dtype=tf.bool))

# encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(next(iterator)[0], next(iterator)[1], max_sequence_length)

In [ ]:
print(encoder_self_attention_mask)
print("==============================")
print(decoder_self_attention_mask)
print("==============================")
print(decoder_cross_attention_mask)

In [ ]:
# import torch
# from torch import nn
# def create_masks(eng_batch, kn_batch):
#     num_sentences = len(eng_batch)
#     look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
#     look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
#     encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
#     decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
#     decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

#     for idx in range(num_sentences):
#       eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
#       eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
#       kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
#       encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
#       encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
#       decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
#       decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
#       decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
#       decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

#     encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
#     decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
#     decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
#     return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

# encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(batch_list[0][0], batch_list[0][1])

In [ ]:
# print(encoder_self_attention_mask)
# print("==============================")
# print(decoder_self_attention_mask)
# print("==============================")
# print(decoder_cross_attention_mask)

In [ ]:
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
num_epochs = 10

# Define loss function with ignore_index equivalent
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
optim= tf.keras.optimizers.Adam(learning_rate=0.001)

# Define a custom loss function to ignore padding tokens
def loss_function(real, pred):
    mask = tf.math.not_equal(real, hindi_to_index[PADDING_TOKEN])
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)


for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    total_loss = 0
    for batch_num, batch in enumerate(iterator):
        eng_batch = batch[0]
        hi_batch = batch[1]
        print(batch_num)
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hi_batch, max_sequence_length)

        with tf.GradientTape() as tape:
            hi_predictions = transformer(eng_batch,
                                         hi_batch,
                                         encoder_self_attention_mask,
                                         decoder_self_attention_mask,
                                         decoder_cross_attention_mask,
                                         enc_start_token=False,
                                         enc_end_token=False,
                                         dec_start_token=True,
                                         dec_end_token=True)
            labels = transformer.decoder.sentence_embedding.batch_tokenize(hi_batch, start_token=False, end_token=True)
            loss = loss_function(labels, hi_predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optim.apply_gradients(zip(gradients, transformer.trainable_variables))
        total_loss += loss.numpy()

        if batch_num % 100 == 0:
            print(f"Batch {batch_num} Loss {total_loss.numpy()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hi_batch[0]}")
            hi_sentence_predicted = tf.argmax(hi_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hi_sentence_predicted:
                if idx == hindi_to_index[END_TOKEN]:
                    break
                predicted_sentence += index_to_hindi[idx.numpy()]
            print(f"Hindi Prediction: {predicted_sentence}")

            transformer.eval()
            hi_sentence = ""
            eng_sentence = "should we go to the mall?"
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks([eng_sentence], [hi_sentence], max_sequence_length)
                predictions = transformer([eng_sentence, hi_sentence, encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask], training=False)
                next_token_prob_distribution = predictions[0][word_counter]
                next_token_index = tf.argmax(next_token_prob_distribution).numpy()
                next_token = index_to_hindi[next_token_index]
                hi_sentence += next_token
                if next_token == END_TOKEN:
                    break

            print(f"Evaluation translation (should we go to the mall?): {hi_sentence}")
            print("-------------------------------------------")


Epoch 1
0
